In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import ModelCheckpoint # type: ignore

In [ ]:
df = pd.read_csv('Data_In.csv')
df.rename(columns = {'company_name' : 'Company'}, inplace = True)

# Print Rows with Null Values
# df[df.isnull().any(axis=1)]

In [ ]:
search_term = 'AMAZON'
factor = 'Company'

# Filter rows where 'Type' is equal to 'Health'
filtered_df = df[df[factor] == search_term]


In [ ]:
# Reshape data into a 1D array
# Scale data to the min and max of the rows i.e 0 -1 range
# Flatten the data once again to a 1D array
# ====================================================================

data = filtered_df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled2 = data_scaled.flatten()


In [ ]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length, 0]
        label = data[i + seq_length, 0]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [ ]:
seq_length = 15
n_samples = len(data_scaled)
sequences, labels = create_sequences(data_scaled, seq_length)
split = int(0.8 * len(sequences))
X_test, y_test = sequences[split:], labels[split:]
X_train, y_train = sequences[:split], labels[:split]

In [49]:
model_lstm = Sequential()
model_lstm.add(LSTM(100, activation='relu', input_shape=(seq_length, 1), return_sequences=True))
model_lstm.add(LSTM(50, activation='relu'))
model_lstm.add(Dense(20))
model_lstm.add(Dense(10))
model_lstm.add(Dense(5))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/home/allan/.virtualenvs/DSC/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# model_lstm.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))
predictions_lstm = model_lstm.predict(X_test)

In [45]:
predictor = load_model('model_lstm.keras')
amazn = predictor.predict(X_test)

/home/allan/.virtualenvs/DSC/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step


In [46]:
amazn = scaler.inverse_transform(amazn)
amazn_y = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
predictions__LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

In [48]:
dates_test = df['Date'].iloc[split + seq_length:]
# df_plot_lstm = pd.DataFrame({'Fechas': dates_test, 'Real': y_test_actual_LSTM.flatten(), 'Predict': predictions__LSTM.flatten()})
df_plot_lstm = pd.DataFrame({'Fechas': dates_test[:len(amazn)], 'Real': amazn_y.flatten(), 'Predict': amazn.flatten()})

df_plot_lstm.tail(50)
# df_plot_lstm.to_csv('AMAZON.csv')

,Fechas,Real,Predict
6651,2007-04-24,128.399994,127.138817
6652,2007-04-25,125.169998,126.209953
6653,2007-04-26,126.559998,124.317299
6654,2007-04-27,128.559998,123.405861
6655,2007-04-30,121.389999,123.700485
6656,2007-05-01,119.570000,121.765984
6657,2007-05-02,127.739998,119.398857
6658,2007-05-03,132.710007,120.374908
6659,2007-05-04,133.089996,123.126450
6660,2007-05-07,137.000000,125.545403
